<a href="https://colab.research.google.com/github/manjeshsinghh/NLP-project/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch nltk rouge streamlit pandas numpy


In [ ]:
# Load the dataset
import pandas as pd
import os

# Use relative path that works in both Colab and local environment
data_path = 'amazon.csv.zip'
if not os.path.exists(data_path):
    # Try Colab path
    data_path = '/content/amazon.csv.zip'

df = pd.read_csv(data_path, compression='zip' if data_path.endswith('.zip') else None)
print(f"Dataset loaded successfully! Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Initialize the GPT-2 model and tokenizer
model_name = "gpt2"  # Using the base GPT-2 model
print(f"Loading model {model_name}...")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set pad_token to eos_token to avoid warnings
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model.to(device)
model.eval()  # Set to evaluation mode

# Initialize ROUGE scorer and BLEU smoothing
rouge = Rouge()
smoothing = SmoothingFunction().method1

print("Model loaded successfully!\n")

# Function to generate text with sampling
def generate_text(prompt, max_new_tokens=150, temperature=0.7, top_k=50, top_p=0.95):
    """
    Generate text using GPT-2 model
    
    Args:
        prompt: Input text prompt
        max_new_tokens: Maximum number of tokens to generate
        temperature: Sampling temperature
        top_k: Top-k sampling
        top_p: Top-p (nucleus) sampling
    
    Returns:
        Generated text (only the new part, excluding the prompt)
    """
    try:
        # Encode the prompt
        inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
        
        # Ensure the length of input doesn't exceed GPT-2's maximum
        max_input_length = 1024
        input_length = inputs.shape[1]
        
        if input_length > max_input_length:
            inputs = inputs[:, -max_input_length:]
        
        # Generate text
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                do_sample=True,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask=torch.ones(inputs.shape, device=device)
            )
        
        # Decode the full output
        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract only the newly generated part (remove the prompt)
        if full_text.startswith(prompt):
            generated_text = full_text[len(prompt):].strip()
        else:
            generated_text = full_text.strip()
        
        return generated_text
    except Exception as e:
        print(f"Error generating text: {str(e)}")
        return ""

# Reward function using BLEU and ROUGE
def reward_function(generated_text, reference_text):
    """
    Calculate reward using BLEU and ROUGE scores
    
    Returns:
        Dictionary with individual and combined scores
    """
    try:
        # Calculate BLEU score with smoothing
        reference_tokens = reference_text.split()
        generated_tokens = generated_text.split()
        
        if len(reference_tokens) == 0 or len(generated_tokens) == 0:
            bleu_score = 0.0
        else:
            bleu_score = sentence_bleu(
                [reference_tokens],
                generated_tokens,
                smoothing_function=smoothing
            )
        
        # Calculate ROUGE scores
        try:
            rouge_scores = rouge.get_scores(generated_text, reference_text)
            rouge_l_score = rouge_scores[0]['rouge-l']['f']
            rouge_1_score = rouge_scores[0]['rouge-1']['f']
            rouge_2_score = rouge_scores[0]['rouge-2']['f']
        except:
            rouge_l_score = 0.0
            rouge_1_score = 0.0
            rouge_2_score = 0.0
        
        # Combine BLEU and ROUGE-L scores
        combined_score = 0.5 * bleu_score + 0.5 * rouge_l_score
        
        return {
            'bleu': bleu_score,
            'rouge_1': rouge_1_score,
            'rouge_2': rouge_2_score,
            'rouge_l': rouge_l_score,
            'combined': combined_score
        }
    except Exception as e:
        print(f"Error calculating reward: {str(e)}")
        return {
            'bleu': 0.0,
            'rouge_1': 0.0,
            'rouge_2': 0.0,
            'rouge_l': 0.0,
            'combined': 0.0
        }

# Iterative feedback loop (without manual input for notebook)
def iterative_feedback(prompt, reference_text, iterations=5, manual_scores=None):
    """
    Iterative feedback loop for text generation
    
    Args:
        prompt: Input prompt
        reference_text: Reference text for evaluation
        iterations: Number of iterations
        manual_scores: List of manual scores (optional, for automated testing)
    """
    results = []
    
    for i in range(iterations):
        # Generate text
        generated_text = generate_text(prompt)
        print(f"\nIteration {i + 1}:")
        print(f"Generated Text:\n{generated_text}\n")
        
        # Evaluate feedback using automatic metrics (BLEU and ROUGE)
        reward_scores = reward_function(generated_text, reference_text)
        print(f"Automatic Metrics:")
        print(f"  BLEU Score: {reward_scores['bleu']:.4f}")
        print(f"  ROUGE-1: {reward_scores['rouge_1']:.4f}")
        print(f"  ROUGE-2: {reward_scores['rouge_2']:.4f}")
        print(f"  ROUGE-L: {reward_scores['rouge_l']:.4f}")
        print(f"  Combined Score: {reward_scores['combined']:.4f}")
        
        # Use manual score if provided, otherwise use a default based on combined score
        if manual_scores and i < len(manual_scores):
            manual_score = manual_scores[i]
        else:
            # Auto-generate a score based on combined metric (for demo purposes)
            manual_score = min(10, max(1, reward_scores['combined'] * 10))
        
        print(f"Manual Feedback Score (simulated): {manual_score:.1f}/10\n")
        
        # Combine the automatic reward and manual feedback
        combined_reward = (0.7 * manual_score / 10) + (0.3 * reward_scores['combined'])
        print(f"Combined Reward (Manual + Automatic): {combined_reward:.4f}\n")
        
        # Store results
        results.append({
            'iteration': i + 1,
            'generated_text': generated_text,
            'reward_scores': reward_scores,
            'manual_score': manual_score,
            'combined_reward': combined_reward
        })
        
        print("-" * 50)
    
    return results

# Example usage (using review content as a pseudo-reference)
print("=" * 50)
print("PROCESSING PRODUCTS")
print("=" * 50)

# Process first 3 products for demonstration
for index, row in df.head(3).iterrows():
    try:
        product_name = row['product_name']
        description = row.get('about_product', 'N/A')
        reference_text = row.get('review_content', 'N/A')
        
        if pd.isna(description) or pd.isna(reference_text):
            print(f"\nSkipping product {index}: Missing data")
            continue
        
        prompt = f"Product Name: {product_name}\nDescription: {description}\nGenerate a compelling product description:"
        
        print(f"\n{'='*50}")
        print(f"Processing product {index + 1}: {product_name[:50]}...")
        print(f"{'='*50}")
        
        # Run iterative feedback (with simulated manual scores)
        results = iterative_feedback(prompt, reference_text, iterations=3)
        
        print(f"\nCompleted processing product {index + 1}")
        
    except Exception as e:
        print(f"Error processing product {index}: {str(e)}")
        continue

print("\n" + "=" * 50)
print("PROCESSING COMPLETE")
print("=" * 50)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Processing product: Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)



/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Iteration 1:
Generated Text:
Product Name: Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)
Description: High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRAN